In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import json
import math
import pandas as pd
import numpy as np
import mapclassify as mc
import geoplot as gplt

import shapely
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from mpl_toolkits import axes_grid1
import matplotlib.cm as cm
import matplotlib.colors as colors

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
color_dict = {"beccs": "#182B49", "bio": "#182B49", # UCSD navy
              "hydro": "#00C6D7", # Turquoise blue
              "nuclear": "#D462AD", # Magenta purple
              "chp": "maroon", "chp_ccs": "coral", 
              "coal_unabated": "#747678", "coal": "#747678", "coal_ccs": "#B6B1A9", # Cool gray and stone
              "gas_unabated": "#F5F0E6", "gas": "#F5F0E6", "gas_ccs": "#84754e", # Metallic and sand
              "wind": "#00629B", "solar": "#C69214", # UCSD blue and UCSD gold
              "Wind": "#00629B", "Solar": "#C69214", # UCSD blue and UCSD gold
              "Offshore_wind": "#00C6D7", "Onshore_wind": "#00629B",
              "Distributed_solar": "#F3E500", "Utility-solar": "#C69214",
              "Offshore wind": "#00C6D7", "Onshore wind": "#00629B",
              "Distributed solar": "#F3E500", "Utility-scale solar": "#C69214",
              "storage": "#6E963B", # Green
              "phs": "#6E963B", "battery": "darkgreen", "lds": "#B6B1A9",
              "Pumped hydro": "#6E963B", "Battery": "darkgreen", 
              "Long duration (CAES)": "#B6B1A9", "Long duration (VRB)": "#B6B1A9",
              "demand": "#00629B", "emission": "#C69214", # UCSD blue and UCSD gold
             }

font_dict = {"family": "Arial",
             "size": 16,
             "color": "#000000"}

color_order = ["#182B49", "#00629B", "#FFCD00", "#C69214", "#6E963B", "#FC8900", "#00C6D7"]
color_order_new = ["#C69214", "#182B49", "#00629B", "#FFCD00", "#6E963B", "#FC8900", "#00C6D7"]
color_order_rgba = ["rgba(198,146,20,0.5)", "rgba(24,43,73,0.3)", 
                    "rgba(0,98,155,0.5)", "rgba(255,205,0,0.5)"]


In [ ]:
result_path = "/Users/zhenhua/Documents/china-re-pathways/LinearOpt_UCSD/data_res"
save_fig_path = "/Users/zhenhua/Documents/1_Project_Pathways/Code_public_final/AdvAppliedEnergy_Pathways_2025/figures_for_papers/figures"


# Fig 2a - Nationwide capacity mix

In [ ]:
# Plot - Figure 2a
fig = make_subplots(rows=1, cols=4, shared_yaxes=True,
                    subplot_titles=("Scenario 2C Baseline", "Scenario 1.5C", "Scenario 2C without CCS", "Scenario 2C with Heat pumps"))

for i in [0, 1, 2, 3]:
    
    res_tag = ["test_0224_365days_all_years_b1_low", "test_0218_365days_all_years_b2_low",
                "test_0315_365days_all_years_b3", "test_0312_365days_all_years_b4"][i]
    vre_year = "w2015_s2015"
    years = [2030, 2040, 2050, 2060]
    
    path = f"{result_path}/{res_tag}_{vre_year}/"
    
    stat_national = pd.read_csv(path + "stat_national.csv").set_index("item")

    # Plot
    legend_sign = True if i == 0 else False
    fig.add_trace(go.Bar(x=years, y=np.asarray(stat_national.loc["offshore_gw"], dtype=float), name="Offshore wind", #text=np.round(res["Offshore_wind"], 2), 
                         legendgroup="group1", showlegend=legend_sign,
                         marker_color="#00C6D7"), row=1, col=i+1)
    fig.add_trace(go.Bar(x=years, y=np.asarray(stat_national.loc["onshore_gw"], dtype=float), name="Onshore wind",#text=np.round(res["Onshore_wind"], 2),
                         legendgroup="group1", showlegend=legend_sign,
                         marker_color=color_dict["Wind"]), row=1, col=i+1)
    fig.add_trace(go.Bar(x=years, y=np.asarray(stat_national.loc["distributed_gw"], dtype=float), name="Distributed PV", #text=np.round(res["Distributed_solar"], 2),
                         legendgroup="group1", showlegend=legend_sign,
                         marker_color="#F3E500"), row=1, col=i+1)
    fig.add_trace(go.Bar(x=years, y=np.asarray(stat_national.loc["utility_gw"], dtype=float), name="Utility-scale PV", #text=np.round(res["Utility_solar"], 2),
                         legendgroup="group1", showlegend=legend_sign,
                         marker_color=color_dict["Solar"]), row=1, col=i+1)
    fig.add_trace(go.Bar(x=years, y=np.asarray(stat_national.loc["cap_phs_gw"], dtype=float), name="Pumped hydro", #text=np.round(res["PHS"], 2),
                         legendgroup="group1", showlegend=legend_sign,
                         marker_color=color_dict["phs"]), row=1, col=i+1)
    fig.add_trace(go.Bar(x=years, y=np.asarray(stat_national.loc["cap_bat_gw"], dtype=float), name="Battery", #text=np.round(res["BAT"], 2),
                         legendgroup="group1", showlegend=legend_sign,
                         marker_color=color_dict["battery"]), row=1, col=i+1)
    fig.add_trace(go.Bar(x=years, y=np.asarray(stat_national.loc["cap_caes_gw"], dtype=float) + \
                         np.asarray(stat_national.loc["cap_vrb_gw"], dtype=float), name="Long duration", #text=np.round(res["BAT"], 2),
                         legendgroup="group1", showlegend=legend_sign,
                         marker_color=color_dict["lds"]), row=1, col=i+1)

fig.update_layout(template="simple_white", barmode='stack', # or "group"
                  font=font_dict,
                  margin=dict(l=0, r=0, b=0),
                  uniformtext_minsize=12, uniformtext_mode='hide',
                  title_text=f"(a) Nationwide Installed VRE and Storage Capacities",
                  legend=dict(yanchor="top", y=1.0, xanchor="left", orientation="h"),
                  width=1200, height=600, 
                  yaxis_range=[0, 11000], 
                  yaxis_title="Capacity (GW)")

fig.show()
fig.write_image(f"{save_fig_path}/figure_2a.pdf", scale=12)


# Fig 2b - Nationwide deployment rate

In [ ]:
# Plot - Figure 2b
res_tags = ["test_0224_365days_all_years_b1_low", "test_0218_365days_all_years_b2_low",
            "test_0315_365days_all_years_b3", "test_0312_365days_all_years_b4"]

res_tags_upper = ["test_0315_365days_all_years_b1_very_high", "test_0228_365days_all_years_b2_very_high"]

scen_names = ["2C Baseline", "1.5C", "2C without CCS", "2C with Heat pumps"]

fig = make_subplots(rows=1, cols=3, shared_yaxes=True, 
                    subplot_titles=("Solar", "Wind", "Storage"),
                    horizontal_spacing=0.05)

for i in range(len(res_tags)):
    res_tag = res_tags[i]
    vre_year = "w2015_s2015"
    path = f"{result_path}/{res_tag}_{vre_year}/"

    # Read provincial deployment rates
    stat_pro = pd.read_csv(path + "stat_provincial.csv")

    stat_nation_r_solar = stat_pro.loc[:33, :][
        ["province", "r_solar_2021_30", "r_solar_2031_40", "r_solar_2041_50", "r_solar_2051_60"]].set_index("province")
    fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"], 
                             y=np.asarray(stat_nation_r_solar.loc["National"], dtype=float), 
                             showlegend=True, marker_symbol="circle", marker_size=15,
                             name=scen_names[i], mode="markers+lines", marker_color=color_order_new[i]),
                  row=1, col=1)
    
    stat_nation_r_wind = stat_pro.loc[:33, :][
        ["province", "r_wind_2021_30", "r_wind_2031_40", "r_wind_2041_50", "r_wind_2051_60"]].set_index("province")
    fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"], 
                             y=np.asarray(stat_nation_r_wind.loc["National"], dtype=float), 
                             showlegend=False, marker_symbol="circle", marker_size=15,
                             name=scen_names[i], mode="markers+lines", marker_color=color_order_new[i]),
                  row=1, col=2)

    # Plot storage deployment rates
    storage_rates = [[4.133, 6.279, 29.228, 62.76], 
                     [4.294, 9.546, 28.611, 50.231],
                     [4.133, 29.06, 144.477, 74.6375],
                     [4.133, 7.2, 29.104, 52.324]]
    fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"], 
                             y=storage_rates[i], 
                             showlegend=False, marker_symbol="circle", marker_size=15,
                             name=scen_names[i], mode="markers+lines", marker_color=color_order_new[i]),
                  row=1, col=3)
    
    # Add national rate error bands:
    if i in [0, 1]:
        # Add national rate error bands for solar
        y_lower = np.asarray(stat_nation_r_solar.loc["National"], dtype=float)
        y_upper = [[59.1651, 88.0237, 105.03635, 235.536],  # b1_very_high
                   [88.9811, 80.9663, 117.80765, 233.7721]][i]   # b2_very_high
        fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"],  y=y_upper,
                                 showlegend=False, line=dict(width=0),
                                 mode="lines", marker_color=color_order[i]),
                      row=1, col=1)
        fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"],  y=y_lower,
                                 showlegend=False, fill='tonexty', fillcolor=color_order_rgba[i],
                                 line=dict(width=0), mode="lines", marker_color=color_order[i]),
                      row=1, col=1)
        # Add national rate error bands for wind
        y_lower = np.asarray(stat_nation_r_wind.loc["National"], dtype=float)
        y_upper = [[48.5922, 92.8062, 54.7524, 130.6212],  # b1_very_high
                   [85.2064, 70.9101, 69.2814, 136.73895]][i]   # b2_very_high
        fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"],  y=y_upper,
                                 showlegend=False, line=dict(width=0),
                                 mode="lines", marker_color=color_order[i]),
                      row=1, col=2)
        fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"],  y=y_lower,
                                 showlegend=False, fill='tonexty', fillcolor=color_order_rgba[i],
                                 line=dict(width=0), mode="lines", marker_color=color_order[i]),
                      row=1, col=2)
        # Add national rate error bands for storage
        y_lower = storage_rates[i]
        y_upper = [[4.132, 21.031, 20.947, 74.0115],  # b1_very_high
                   [4.294, 27.846, 19.31, 63.943]][i]   # b2_very_high
        fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"],  y=y_upper,
                                 showlegend=False, line=dict(width=0),
                                 mode="lines", marker_color=color_order[i]),
                      row=1, col=3)
        fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"],  y=y_lower,
                                 showlegend=False, fill='tonexty', fillcolor=color_order_rgba[i],
                                 line=dict(width=0), mode="lines", marker_color=color_order[i]),
                      row=1, col=3)

# Plot current deployment rates
fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"],  y=[119] * 4, line_width=5,
                         showlegend=False, marker_symbol="x", marker_size=15, line_dash="dash",
                         name="2021-2023 Rates", mode="lines", marker_color="#D462AD"), row=1, col=1)
fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"],  y=[53] * 4, line_width=5,
                         showlegend=False, marker_symbol="x", marker_size=15, line_dash="dash",
                         name="2021-2023 Rates", mode="lines", marker_color="#D462AD"), row=1, col=2)
fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"],  y=[28] * 4, line_width=5,
                         showlegend=False, marker_symbol="x", marker_size=15, line_dash="dash",
                         name="Historical Rates (2021-2023)", mode="lines", marker_color="#D462AD"), row=1, col=3)
# Code to add the text annotation below the line
fig.add_annotation(text="Historical Rates<br>(2021-2023)",
                   y=100,     # Anchor the annotation to the line's y-coordinate
                   yshift=-10, # Shift the text 10 pixels down from the anchor point
                   showarrow=False, font=dict(color="#D462AD"  # Specify the text color here
                   ),
                   row=1, col=1)

fig.update_layout(template="simple_white", boxmode='group',
                  font=font_dict,
                  margin=dict(l=0, r=0, b=0),
                  uniformtext_minsize=12, uniformtext_mode='hide',
                  title=f"(b) Nationwide VRE and Storage Deployment Rates",
                  legend=dict(yanchor="top", y=1.0, xanchor="left", x=0, orientation="v", traceorder="normal"),
                  width=1200, height=600, 
                  yaxis_range=[0, 300], yaxis_title="Deployment Rate (GW/year)")

fig.show()  
fig.write_image(f"{save_fig_path}/figure_2b.pdf", scale=12)


# Fig 3 - Regional deployment rates

In [ ]:
# Plot - Figure 3: group by GRID REGION

scen_names = ["2C Baseline", "1.5C", "2C without CCS", "2C with Heat pumps"]

fig = make_subplots(rows=1, cols=6, shared_yaxes=True, shared_xaxes=False, 
                    subplot_titles=("East", "Central", "South", "North", "Northeast", "Northwest")
                   )

for i in [0, 1, 2, 3]:
    res_tag = ["test_0224_365days_all_years_b1_low", "test_0218_365days_all_years_b2_low",
               "test_0315_365days_all_years_b3", "test_0312_365days_all_years_b4"][i]
    vre_year = "w2015_s2015"

    path = f"{result_path}/{res_tag}_{vre_year}/"
    pro_meta = pd.read_csv("/Users/zhenhua/Documents/china-re-pathways/LinearOpt_UCSD/data_csv/geography/China_provinces_hz.csv")
    pro_meta = pro_meta[["provin_py", "grid"]]

    # Solar
    stat_pro = pd.read_csv(path + "stat_provincial.csv")
    stat_pro = stat_pro.merge(pro_meta, left_on="province", right_on="provin_py", how="left")
    stat_pro_r_solar = stat_pro.loc[:31, :][
        ["province", "grid", "r_solar_2021_30", "r_solar_2031_40", "r_solar_2041_50", "r_solar_2051_60"]]
    for col in ["r_solar_2021_30", "r_solar_2031_40", "r_solar_2041_50", "r_solar_2051_60"]:
        stat_pro_r_solar[col] = np.asarray(stat_pro_r_solar[col], dtype=float)
    
    # Wind
    stat_pro_r_wind = stat_pro.loc[:31, :][
        ["province", "r_wind_2021_30", "r_wind_2031_40", "r_wind_2041_50", "r_wind_2051_60"]]
    for col in ["r_wind_2021_30", "r_wind_2031_40", "r_wind_2041_50", "r_wind_2051_60"]:
        stat_pro_r_wind[col] = np.asarray(stat_pro_r_wind[col], dtype=float)
    
    stat_pro_r_vre = pd.merge(stat_pro_r_solar, stat_pro_r_wind, on="province")
    stat_pro_r_vre["r_vre_2021_30"] = stat_pro_r_vre["r_solar_2021_30"] + stat_pro_r_vre["r_wind_2021_30"]
    stat_pro_r_vre["r_vre_2031_40"] = stat_pro_r_vre["r_solar_2031_40"] + stat_pro_r_vre["r_wind_2031_40"]
    stat_pro_r_vre["r_vre_2041_50"] = stat_pro_r_vre["r_solar_2041_50"] + stat_pro_r_vre["r_wind_2041_50"]
    stat_pro_r_vre["r_vre_2051_60"] = stat_pro_r_vre["r_solar_2051_60"] + stat_pro_r_vre["r_wind_2051_60"]
    
    # Future VRE combined deployment rates
    stat_grid_r_vre = stat_pro_r_vre.groupby("grid").sum()
    stat_grid_r_vre = stat_grid_r_vre.T
    stat_grid_r_vre = stat_grid_r_vre.loc[["r_vre_2021_30", "r_vre_2031_40", "r_vre_2041_50", "r_vre_2051_60"]]
    # stat_grid_r_vre
    
    # Read current deployment rates
    deploy_r_solar = pd.read_excel("/Users/zhenhua/Documents/china-re-pathways/LinearOpt_UCSD/data_csv/vre_installations/vre_provincial_202403.xlsx")
    deploy_r_solar = deploy_r_solar.loc[:30, :]
    deploy_grid_r_solar = deploy_r_solar[["grid", "solar_delta_2019_2022"]].groupby("grid").sum()
    
    deploy_r_wind = pd.read_excel("/Users/zhenhua/Documents/china-re-pathways/LinearOpt_UCSD/data_csv/vre_installations/vre_provincial_202403.xlsx")
    deploy_r_wind = deploy_r_wind.loc[:30, :]
    deploy_grid_r_wind = deploy_r_wind[["grid", "wind_delta_2019_2022"]].groupby("grid").sum()
    
    deploy_grid_r_vre = pd.merge(deploy_grid_r_solar, deploy_grid_r_wind, on="grid")
    deploy_grid_r_vre["vre_delta_2019_2022"] = deploy_grid_r_vre["solar_delta_2019_2022"] + deploy_grid_r_vre["wind_delta_2019_2022"]
    # deploy_grid_r_vre

    # Plot requirements and deployment rates by grid region
    for j in range(0,6):
        legend_sign = True if j == 0 else False
        r = ["EC", "CC", "SC", "NC", "NE", "NW"][j]        
        fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"], y=stat_grid_r_vre[r],
                                 orientation='h', name=scen_names[i], mode="markers+lines",
                                 legendgroup="group1", showlegend=legend_sign,
                                 marker_symbol="square", marker_size=15, marker_color=color_order_new[i],
                                ), row=1, col=j+1)
        
        # Plot current deployment rates
        if i == 3:
            fig.add_trace(go.Scatter(x=["2030", "2040", "2050", "2060"], y=[deploy_grid_r_vre.loc[r, "vre_delta_2019_2022"]] * 4, line_width=5,
                                     showlegend=False, marker_symbol="x", marker_size=15, line_dash="dash",
                                     name="Historical Rates (2018-2022)", mode="lines", marker_color="#D462AD"), row=1, col=j+1)
    
# Code to add the text annotation below the line
fig.add_annotation(text="Historical Rates<br>(2018-2022)",
                   x=1.8, y=20,     # Anchor the annotation to the line's y-coordinate
                   yshift=-50, # Shift the text 10 pixels down from the anchor point
                   showarrow=False, font=dict(color="#D462AD"  # Specify the text color here
                   ),
                   row=1, col=1)

fig.update_layout(template="simple_white", barmode='overlay', # or "group"
                  font=font_dict,
                  margin=dict(l=0, r=0, b=0, t=20),
                  uniformtext_minsize=9, uniformtext_mode='show',
#                   title=f"Subnational VRE Deployment Rates",
                  legend=dict(yanchor="top", y=1.0, xanchor="left", x=0, orientation="v", itemwidth=30),
                  width=1200, height=600, 
                  yaxis_title="Deployment Rate (GW/year)",
                  yaxis_range=[0, 120], 
                 )

fig.show()
fig.write_image(f"{save_fig_path}/figure_3.pdf", scale=12)


# Fig 4 - Cell-level VRE capacity

In [ ]:
# Specify file paths
work_dir = "/Users/zhenhua/Documents/china-re-pathways/LinearOpt_UCSD/"
res_tag = "test_0822_365days_all_years_b1"
vre_year = "w2015_s2015"
dir_flag = "/"

vre_type = "Utility-scale PV"
# vre_type = "Distributed PV"
# vre_type = "Onshore and Offshore Wind"

cmap = 'PuBuGn'
base_cmap = plt.get_cmap(cmap)
cmap_new_capacity = base_cmap(np.linspace(0.2, 1.0, base_cmap.N))
cmap_new_capacity = colors.ListedColormap(cmap_new_capacity)

def getBound():
    bound = gpd.GeoDataFrame({
    'x': [68, 140, 106.5, 123],
    'y': [15, 55, 2.8, 24.5]
    })
    bound.geometry = bound.apply(lambda row: Point([row['x'], row['y']]), axis=1)
    bound.crs = 'EPSG:4326'
    return bound

fig, axes = plt.subplots(4, 1, figsize=(6, 20), sharex=False, sharey=False)

for i in [0, 1, 2, 3]:
    # Determine which subplot
    ax = axes[i]
    curr_year = [2030, 2040, 2050, 2060][i]

    out_path = os.path.join(work_dir, "data_res", f"{res_tag}_{vre_year}", str(curr_year))
    out_input_path = os.path.join(out_path, "inputs")
    out_output_path = os.path.join(out_path, "outputs")
    out_output_processed_path = os.path.join(out_path, "outputs_processed")

    # Read outputs
    vre_shp = {'new': [], 'polygon': [], 'ins_and_new': []}
    
    vre_info = open(os.path.join(out_output_processed_path, 'solar_info.csv'), 'r+')
    for line in vre_info:
        line = line.replace('\n', '')
        line = line.split(',')
        
        if vre_type == "Utility-scale PV":
            
            if (eval(line[1]) > 0) and (eval(line[9]) == 0):
                new_cap = (eval(line[1]) - eval(line[10])) * eval(line[4])
                IN_cap = eval(line[1]) * eval(line[4])

                vre_shp['new'].append(new_cap)
                vre_shp['ins_and_new'].append(IN_cap)
                x = eval(line[2])  # lon
                y = eval(line[3])  # lat

                vre_shp['polygon'].append(
                    shapely.geometry.Polygon(
                        [(x - 0.15625, y - 0.125),
                         (x + 0.15625, y - 0.125),
                         (x + 0.15625, y + 0.125),
                         (x - 0.15625, y + 0.125)]
                    )
                )

    vre_gdf = gpd.GeoDataFrame(vre_shp, geometry=vre_shp['polygon'])

    china = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'china_country.shp')
    china_grids = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'grids' + dir_flag + 'Grid_regions_four.shp')
    china_nine_lines = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'china_nine_dotted_line.shp')

    bound = getBound()

    # Change the default font family
    font = {'family': 'Arial', 'size': 24}
    plt.rcParams.update({'font.family': font["family"]})

    # New capacity
    divider = axes_grid1.make_axes_locatable(ax)
    cax = divider.append_axes('right', size="5%", pad=0.4)
        
    ax = vre_gdf.plot(column="new", ax=ax, legend=True, cax=cax, cmap=cmap_new_capacity,
                      edgecolor='none', linewidth=0.0, vmin=0, vmax=1)

    ax = china_nine_lines.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', linewidth=0.5)
    ax = china.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', linewidth=0.4)
    ax = china_grids.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', alpha=0, linewidth=0.5)

    ax.axis('off')
    ax.set_xlim(bound.geometry[0].x, bound.geometry[1].x)
    ax.set_ylim(bound.geometry[0].y, bound.geometry[1].y)

    last_year = curr_year - 10
    ax.set_title(f'{last_year}-{curr_year}', font=font, y=0.95)
    
    # New capacity in ax_child
    ax_child = ax.inset_axes((0.75, 0, 0.3, 0.3))
    
    # Set color and linewidth for the ax_child frame
    for spine in ax_child.spines.values():
        spine.set_edgecolor('#747678') # Set the color of the box
        spine.set_linewidth(0.5)     # Set the linewidth of the box (in points)
        
    vre_gdf.plot(column="new", ax=ax_child, legend=False, cmap=cmap_new_capacity,
                    edgecolor='none', linewidth=0.0, vmin=0, vmax=1)

    ax_child = china_nine_lines.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', linewidth=0.3)
    ax_child = china.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', linewidth=0.2)
    ax_child = china_grids.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', alpha=0, linewidth=0.3)

    ax_child.set_xlim(bound.geometry[2].x, bound.geometry[3].x)
    ax_child.set_ylim(bound.geometry[2].y, bound.geometry[3].y)
    ax_child.set_xticks([])
    ax_child.set_yticks([])

plt.subplots_adjust(wspace=0.0) # Reduced horizontal space (0.05 is a small fraction of subplot width)
plt.subplots_adjust(hspace=0)
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Add a main title for the whole figure (super title)
fig.suptitle(f"(a) {vre_type}", fontsize=24, y=1.05, fontweight="bold")

# Add a main colorbar for the whole figure
# Create a ScalarMappable to link the colorbar to the data range and colormap
# This uses the vmin/vmax (0 to 1) and cmap_new_capacity defined for the plots.
mappable = cm.ScalarMappable(norm=colors.Normalize(vmin=0, vmax=1), cmap=cmap_new_capacity)
mappable.set_array([]) # Important: set an empty array for the mappable
# Create a new axes for the main colorbar. Position it below the suptitle.
# [left, bottom, width, height] as fractions of the figure width/height
cax_main = fig.add_axes([0.1, 1.0, 0.6, 0.015]) # Adjusted position and size for the main colorbar
# Add the colorbar to the main axes
cbar_main = fig.colorbar(mappable, cax=cax_main, orientation='horizontal')
cbar_main.ax.tick_params(labelsize=16) # Adjust tick label size for main colorbar

# Add text annotation right next to the main horizontal colorbar
cax_main.text(1.25, 1.0, 'Capacity (GW)', transform=cax_main.transAxes,
              ha='center', va='top', fontsize=16, color='black')


plt.show()
fig.savefig(f"{save_fig_path}/figure_4_2C_upv.pdf", dpi=300, bbox_inches='tight')


In [ ]:
# Specify file paths
work_dir = "/Users/zhenhua/Documents/china-re-pathways/LinearOpt_UCSD/"
res_tag = "test_0822_365days_all_years_b1"
vre_year = "w2015_s2015"
dir_flag = "/"

# vre_type = "Utility-scale PV"
vre_type = "Distributed PV"
# vre_type = "Onshore and Offshore Wind"

# cmap = 'PuBuGn'
cmap = 'YlOrBr'
base_cmap = plt.get_cmap(cmap)
cmap_new_capacity = base_cmap(np.linspace(0.2, 1.0, base_cmap.N))
cmap_new_capacity = colors.ListedColormap(cmap_new_capacity)

fig, axes = plt.subplots(4, 1, figsize=(6, 20), sharex=False, sharey=False)

for i in [0, 1, 2, 3]:
    # Determine which subplot
    ax = axes[i]
    curr_year = [2030, 2040, 2050, 2060][i]

    out_path = os.path.join(work_dir, "data_res", f"{res_tag}_{vre_year}", str(curr_year))
    out_input_path = os.path.join(out_path, "inputs")
    out_output_path = os.path.join(out_path, "outputs")
    out_output_processed_path = os.path.join(out_path, "outputs_processed")

    # Read outputs
    vre_shp = {'new': [], 'polygon': [], 'ins_and_new': []}
    

    vre_info = open(os.path.join(out_output_processed_path, 'solar_info.csv'), 'r+')
    for line in vre_info:
        line = line.replace('\n', '')
        line = line.split(',')
        
        if vre_type == "Distributed PV":
            
            if (eval(line[1]) > 0) and (eval(line[9]) == 1):
                new_cap = (eval(line[1]) - eval(line[10])) * eval(line[4])
                IN_cap = eval(line[1]) * eval(line[4])

                vre_shp['new'].append(new_cap)
                vre_shp['ins_and_new'].append(IN_cap)
                x = eval(line[2])  # lon
                y = eval(line[3])  # lat

                vre_shp['polygon'].append(
                    shapely.geometry.Polygon(
                        [(x - 0.15625, y - 0.125),
                         (x + 0.15625, y - 0.125),
                         (x + 0.15625, y + 0.125),
                         (x - 0.15625, y + 0.125)]
                    )
                )

    vre_gdf = gpd.GeoDataFrame(vre_shp, geometry=vre_shp['polygon'])

    china = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'china_country.shp')
    china_grids = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'grids' + dir_flag + 'Grid_regions_four.shp')
    china_nine_lines = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'china_nine_dotted_line.shp')

    bound = getBound()

    # Change the default font family
    font = {'family': 'Arial', 'size': 24}
    plt.rcParams.update({'font.family': font["family"]})

    # New capacity
    divider = axes_grid1.make_axes_locatable(ax)
    cax = divider.append_axes('right', size="5%", pad=0.4)
        
    ax = vre_gdf.plot(column="new", ax=ax, legend=True, cax=cax, cmap=cmap_new_capacity,
                      edgecolor='none', linewidth=0.0, vmin=0, vmax=1)

    ax = china_nine_lines.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', linewidth=0.5)
    ax = china.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', linewidth=0.4)
    ax = china_grids.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', alpha=0, linewidth=0.5)

    ax.axis('off')
    ax.set_xlim(bound.geometry[0].x, bound.geometry[1].x)
    ax.set_ylim(bound.geometry[0].y, bound.geometry[1].y)

    last_year = curr_year - 10
    ax.set_title(f'{last_year}-{curr_year}', font=font, y=0.95)
    
    # New capacity in ax_child
    ax_child = ax.inset_axes((0.75, 0, 0.3, 0.3))
    
    # Set color and linewidth for the ax_child frame
    for spine in ax_child.spines.values():
        spine.set_edgecolor('#747678') # Set the color of the box
        spine.set_linewidth(0.5)     # Set the linewidth of the box (in points)
        
    vre_gdf.plot(column="new", ax=ax_child, legend=False, cmap=cmap_new_capacity,
                    edgecolor='none', linewidth=0.0, vmin=0, vmax=1)

    ax_child = china_nine_lines.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', linewidth=0.3)
    ax_child = china.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', linewidth=0.2)
    ax_child = china_grids.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', alpha=0, linewidth=0.3)

    ax_child.set_xlim(bound.geometry[2].x, bound.geometry[3].x)
    ax_child.set_ylim(bound.geometry[2].y, bound.geometry[3].y)
    ax_child.set_xticks([])
    ax_child.set_yticks([])

plt.subplots_adjust(wspace=0.0) # Reduced horizontal space (0.05 is a small fraction of subplot width)
plt.subplots_adjust(hspace=0)
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Add a main title for the whole figure (super title)
fig.suptitle(f"(b) {vre_type}", fontsize=24, y=1.05, fontweight="bold")

# Add a main colorbar for the whole figure
# Create a ScalarMappable to link the colorbar to the data range and colormap
# This uses the vmin/vmax (0 to 1) and cmap_new_capacity defined for the plots.
mappable = cm.ScalarMappable(norm=colors.Normalize(vmin=0, vmax=1), cmap=cmap_new_capacity)
mappable.set_array([]) # Important: set an empty array for the mappable
# Create a new axes for the main colorbar. Position it below the suptitle.
# [left, bottom, width, height] as fractions of the figure width/height
cax_main = fig.add_axes([0.1, 1.0, 0.6, 0.015]) # Adjusted position and size for the main colorbar
# Add the colorbar to the main axes
cbar_main = fig.colorbar(mappable, cax=cax_main, orientation='horizontal')
cbar_main.ax.tick_params(labelsize=16) # Adjust tick label size for main colorbar

# Add text annotation right next to the main horizontal colorbar
cax_main.text(1.25, 1.0, 'Capacity (GW)', transform=cax_main.transAxes,
              ha='center', va='top', fontsize=16, color='black')


plt.show()
fig.savefig(f"{save_fig_path}/figure_4_2C_dpv.pdf", dpi=300, bbox_inches='tight')


In [ ]:
# Specify file paths
work_dir = "/Users/zhenhua/Documents/china-re-pathways/LinearOpt_UCSD/"
res_tag = "test_0822_365days_all_years_b1"
vre_year = "w2015_s2015"
dir_flag = "/"

# vre_type = "Utility-scale PV"
# vre_type = "Distributed PV"
vre_type = "Onshore and Offshore Wind"

cmap = 'PuBuGn'
base_cmap = plt.get_cmap(cmap)
cmap_new_capacity = base_cmap(np.linspace(0.2, 1.0, base_cmap.N))
cmap_new_capacity = colors.ListedColormap(cmap_new_capacity)

cmap_2 = 'YlOrBr'
base_cmap_2 = plt.get_cmap(cmap_2)
cmap_new_capacity_2 = base_cmap_2(np.linspace(0.2, 1.0, base_cmap_2.N))
cmap_new_capacity_2 = colors.ListedColormap(cmap_new_capacity_2)

fig, axes = plt.subplots(4, 1, figsize=(6, 20), sharex=False, sharey=False)

for i in [0, 1, 2, 3]:
    # Determine which subplot
    ax = axes[i]
    curr_year = [2030, 2040, 2050, 2060][i]

    out_path = os.path.join(work_dir, "data_res", f"{res_tag}_{vre_year}", str(curr_year))
    out_input_path = os.path.join(out_path, "inputs")
    out_output_path = os.path.join(out_path, "outputs")
    out_output_processed_path = os.path.join(out_path, "outputs_processed")

    # Read outputs
    vre_shp = {'new': [], 'polygon': [], 'ins_and_new': []}
    off_shp = {'new': [], 'polygon': [], 'ins_and_new': []}
    

    vre_info = open(os.path.join(out_output_processed_path, 'wind_info.csv'), 'r+')
    for line in vre_info:
        line = line.replace('\n', '')
        line = line.split(',')
        
        if vre_type == "Onshore and Offshore Wind":
            
            if (eval(line[1]) > 0) and (eval(line[9]) == 1):
                new_cap = (eval(line[1]) - eval(line[10])) * eval(line[4])
                IN_cap = eval(line[1]) * eval(line[4])

                vre_shp['new'].append(new_cap)
                vre_shp['ins_and_new'].append(IN_cap)
                x = eval(line[2])  # lon
                y = eval(line[3])  # lat

                vre_shp['polygon'].append(
                    shapely.geometry.Polygon(
                        [(x - 0.15625, y - 0.125),
                         (x + 0.15625, y - 0.125),
                         (x + 0.15625, y + 0.125),
                         (x - 0.15625, y + 0.125)]
                    )
                )
            
            if (eval(line[1]) > 0) and (eval(line[9]) == 0):
                new_cap = (eval(line[1]) - eval(line[10])) * eval(line[4])
                IN_cap = eval(line[1]) * eval(line[4])

                off_shp['new'].append(new_cap)
                off_shp['ins_and_new'].append(IN_cap)
                x = eval(line[2])  # lon
                y = eval(line[3])  # lat

                off_shp['polygon'].append(
                    shapely.geometry.Polygon(
                        [(x - 0.15625, y - 0.125),
                         (x + 0.15625, y - 0.125),
                         (x + 0.15625, y + 0.125),
                         (x - 0.15625, y + 0.125)]
                    )
                )

    vre_gdf = gpd.GeoDataFrame(vre_shp, geometry=vre_shp['polygon'])
    off_gdf = gpd.GeoDataFrame(off_shp, geometry=off_shp['polygon'])

    china = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'china_country.shp')
    china_grids = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'grids' + dir_flag + 'Grid_regions_four.shp')
    china_nine_lines = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'china_nine_dotted_line.shp')

    bound = getBound()

    # Change the default font family
    font = {'family': 'Arial', 'size': 24}
    plt.rcParams.update({'font.family': font["family"]})

    # New capacity
    divider = axes_grid1.make_axes_locatable(ax)
    cax = divider.append_axes('right', size="5%", pad=0.4)
        
    ax = vre_gdf.plot(column="new", ax=ax, legend=True, cax=cax, cmap=cmap_new_capacity,
                      edgecolor='none', linewidth=0.0, vmin=0, vmax=1)
    off_gdf.plot(column="new", ax=ax, legend=True, cax=cax, cmap=cmap_new_capacity_2,
                  edgecolor='none', linewidth=0.0, vmin=0, vmax=1)

    ax = china_nine_lines.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', linewidth=0.5)
    ax = china.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', linewidth=0.4)
    ax = china_grids.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', alpha=0, linewidth=0.5)

    ax.axis('off')
    ax.set_xlim(bound.geometry[0].x, bound.geometry[1].x)
    ax.set_ylim(bound.geometry[0].y, bound.geometry[1].y)

    last_year = curr_year - 10
    ax.set_title(f'{last_year}-{curr_year}', font=font, y=0.95)
    
    # New capacity in ax_child
    ax_child = ax.inset_axes((0.75, 0, 0.3, 0.3))
    
    # Set color and linewidth for the ax_child frame
    for spine in ax_child.spines.values():
        spine.set_edgecolor('#747678') # Set the color of the box
        spine.set_linewidth(0.5)     # Set the linewidth of the box (in points)
        
    vre_gdf.plot(column="new", ax=ax_child, legend=False, cmap=cmap_new_capacity,
                    edgecolor='none', linewidth=0.0, vmin=0, vmax=1)
    off_gdf.plot(column="new", ax=ax_child, legend=False, cmap=cmap_new_capacity_2,
                    edgecolor='none', linewidth=0.0, vmin=0, vmax=1)

    ax_child = china_nine_lines.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', linewidth=0.3)
    ax_child = china.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', linewidth=0.2)
    ax_child = china_grids.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', alpha=0, linewidth=0.3)

    ax_child.set_xlim(bound.geometry[2].x, bound.geometry[3].x)
    ax_child.set_ylim(bound.geometry[2].y, bound.geometry[3].y)
    ax_child.set_xticks([])
    ax_child.set_yticks([])

plt.subplots_adjust(wspace=0.0) # Reduced horizontal space (0.05 is a small fraction of subplot width)
plt.subplots_adjust(hspace=0)
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Add a main title for the whole figure (super title)
fig.suptitle(f"(c) {vre_type}", fontsize=24, y=1.05, fontweight="bold")

# Add a main colorbar for the whole figure
# Create a ScalarMappable to link the colorbar to the data range and colormap
# This uses the vmin/vmax (0 to 1) and cmap_new_capacity defined for the plots.
mappable = cm.ScalarMappable(norm=colors.Normalize(vmin=0, vmax=1), cmap=cmap_new_capacity)
mappable.set_array([]) # Important: set an empty array for the mappable
cax_main = fig.add_axes([0.1, 1.0, 0.6, 0.015]) # Adjusted position and size for the main colorbar
cbar_main = fig.colorbar(mappable, cax=cax_main, orientation='horizontal')
cbar_main.ax.tick_params(labelsize=16) # Adjust tick label size for main colorbar

cax_main.text(1.25, 1.0, 'Onshore \nCapacity (GW)', transform=cax_main.transAxes,
              ha='center', va='top', fontsize=16, color='black')


mappable = cm.ScalarMappable(norm=colors.Normalize(vmin=0, vmax=1), cmap=cmap_new_capacity_2)
mappable.set_array([]) # Important: set an empty array for the mappable
cax_main = fig.add_axes([0.1, 0.96, 0.6, 0.015]) # Adjusted position and size for the main colorbar
cbar_main = fig.colorbar(mappable, cax=cax_main, orientation='horizontal')
cbar_main.ax.tick_params(labelsize=16) # Adjust tick label size for main colorbar

cax_main.text(1.25, 0.96, 'Offshore \nCapacity (GW)', transform=cax_main.transAxes,
              ha='center', va='top', fontsize=16, color='black')


plt.show()
fig.savefig(f"{save_fig_path}/figure_4_2C_wind.pdf", dpi=300, bbox_inches='tight')


# Fig 5 - Transmission map

In [ ]:
# Specify file paths
work_dir = "/Users/zhenhua/Documents/china-re-pathways/LinearOpt_UCSD/"
res_tag = "test_0822_365days_all_years_b1"
vre_year = "w2015_s2015"
dir_flag = "/"

fig, axes = plt.subplots(2, 2, figsize=(12, 10), sharex=False, sharey=False)

for i in [0, 1, 2, 3]:
    # Determine which subplot
    ax = axes.ravel()[i]
    curr_year = [2030, 2040, 2050, 2060][i]

    out_path = os.path.join(work_dir, "data_res", f"{res_tag}_{vre_year}", str(curr_year))
    out_input_path = os.path.join(out_path, "inputs")
    out_output_path = os.path.join(out_path, "outputs")
    out_output_processed_path = os.path.join(out_path, "outputs_processed")


    newTransCap = {}
    f_ntc = open(os.path.join(out_output_path, 'new_trans_cap', 'cap_trans_new.csv'), 'r+')
    for line in f_ntc:
        line = line.replace('\n', '')
        line = line.split(',')
        line[2] = eval(line[2])
        newTransCap[(line[0], line[1])] = round(line[2], 2)
    f_ntc.close()

    existingTransCap = {}
    f_etc = pd.read_csv(os.path.join(out_input_path, 'inter_pro_trans.csv'))
    for pro in f_etc['province']:
        for i in range(len(f_etc[pro])):
            existingTransCap[(pro, f_etc['province'][i])] = 0.5 * 0.001 * f_etc[pro][i]  # MW -> GW
    f_etc = None

    totalTransCap = {}
    for pro_pair in newTransCap:
        totalTransCap[pro_pair] = newTransCap[pro_pair] + existingTransCap[pro_pair]

    provin_geo = {}
    f_pg = open(work_dir + 'data_csv' + dir_flag + 'geography/China_provinces_hz.csv', 'r+')
    next(f_pg)
    for line in f_pg:
        line = line.replace('\n', '')
        line = line.split(',')
        provin_geo[line[1]] = (eval(line[3]), eval(line[4]))
    f_pg.close()

    new_trans_shp = {'cap': [], 'linestring': []}
    count = []
    for pro_pair in newTransCap:
        if pro_pair not in count:
            count.append((pro_pair[1], pro_pair[0]))
            cap = newTransCap[pro_pair] + newTransCap[(pro_pair[1], pro_pair[0])]
            if cap != 0:
                new_trans_shp['cap'].append(cap)
                new_trans_shp['linestring'].append(
                    shapely.geometry.LineString(
                        [(provin_geo[pro_pair[0]][0], provin_geo[pro_pair[0]][1]),
                         (provin_geo[pro_pair[1]][0], provin_geo[pro_pair[1]][1])]
                    )
                )
    # Add 40 GW as a placeholder for plotting
    new_trans_shp['cap'].append(40)
    new_trans_shp['linestring'].append(shapely.geometry.LineString([(0, 0), (1, 0)]))

    existing_trans_shp = {'cap': [], 'linestring': []}
    count = []
    for pro_pair in existingTransCap:
        if pro_pair not in count:
            count.append((pro_pair[1], pro_pair[0]))
            cap = existingTransCap[pro_pair] + existingTransCap[(pro_pair[1], pro_pair[0])]
            if cap != 0:
                existing_trans_shp['cap'].append(cap)
                existing_trans_shp['linestring'].append(
                    shapely.geometry.LineString(
                        [(provin_geo[pro_pair[0]][0], provin_geo[pro_pair[0]][1]),
                         (provin_geo[pro_pair[1]][0], provin_geo[pro_pair[1]][1])]
                    )
                )

    total_trans_shp = {'cap': [], 'linestring': []}
    count = []
    for pro_pair in totalTransCap:
        if pro_pair not in count:
            count.append((pro_pair[1], pro_pair[0]))
            cap = totalTransCap[pro_pair] + totalTransCap[(pro_pair[1], pro_pair[0])]
            if cap != 0:
                total_trans_shp['cap'].append(cap)
                total_trans_shp['linestring'].append(
                    shapely.geometry.LineString(
                        [(provin_geo[pro_pair[0]][0], provin_geo[pro_pair[0]][1]),
                         (provin_geo[pro_pair[1]][0], provin_geo[pro_pair[1]][1])]
                    )
                )


    # Change the default font family
    font = {'family': 'Arial', 'size': 24}
    plt.rcParams.update({'font.family': font["family"]})

    trans_shp = {
        'New': new_trans_shp,
    }

    china_2 = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'province/china_provinces.shp')
    china = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'province' + dir_flag + 'china_provinces_nmsplit.shp')
    china_grids = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'grids' + dir_flag + 'Grid_regions_four.shp')
    china_nine_lines = gpd.read_file(work_dir + 'data_shp' + dir_flag + 'china_nine_dotted_line.shp')

    for trans_kind in trans_shp:
        trans_gdf = gpd.GeoDataFrame(trans_shp[trans_kind], geometry=trans_shp[trans_kind]['linestring'])
        bound = getBound()

        if trans_kind == 'Existing':
            legend_values = [0.1, 15, 30, 45, 60]
            cmap_vmax = 80
            cmap_name = "autumn_r"
        elif trans_kind == 'Total':
            legend_values = [0.4, 20, 40, 60, 80]
            cmap_vmax = 80
            cmap_name = "autumn_r"
        elif trans_kind == 'New':
            cmap_vmax = 40
            cmap_name = "cividis_r"

        
        scheme = mc.Quantiles(trans_gdf['cap'], k=5)

        gplt.sankey(
            trans_gdf,
            hue='cap',
            scale='cap',
            legend=False,
            cmap=cmap_name,
            ax=ax
        )

        china_nine_lines.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', linewidth=0.5)
        china_2.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', linewidth=0.4)
        china_grids.geometry.plot(ax=ax, edgecolor='#747678', facecolor='None', alpha=0, linewidth=0.5)

        ax.set_xlim(bound.geometry[0].x, bound.geometry[1].x)
        ax.set_ylim(bound.geometry[0].y, bound.geometry[1].y)

        ax.set_title(f'{curr_year-10}-{curr_year}', font=font, y=0.95)

        ax_child = ax.inset_axes((0.75, 0, 0.3, 0.3))

        # Set color and linewidth for the ax_child frame
        for spine in ax_child.spines.values():
            spine.set_edgecolor('#747678') # Set the color of the box
            spine.set_linewidth(0.5)     # Set the linewidth of the box (in points)
            
        gplt.sankey(trans_gdf, hue='cap', scale='cap', legend=False, cmap=cmap_name,
                    ax=ax_child
                    )
        ax_child = china_nine_lines.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', linewidth=0.3)
        ax_child = china.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', linewidth=0.2)
        ax_child = china_grids.geometry.plot(ax=ax_child, edgecolor='#747678', facecolor='None', alpha=0, linewidth=0.3)
        
        ax_child.set_xlim(bound.geometry[2].x, bound.geometry[3].x)
        ax_child.set_ylim(bound.geometry[2].y, bound.geometry[3].y)
        ax_child.set_xticks([])
        ax_child.set_yticks([])
        ax_child.axis('on')

plt.subplots_adjust(wspace=0.0) # Reduced horizontal space (0.05 is a small fraction of subplot width)
plt.subplots_adjust(hspace=0)
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Add a main colorbar for the whole figure
# Create a ScalarMappable to link the colorbar to the data range and colormap
# This uses the vmin/vmax (0 to 1) and cmap_new_capacity defined for the plots.
mappable = cm.ScalarMappable(norm=colors.Normalize(vmin=0, vmax=40), cmap=cmap_name)
mappable.set_array([]) # Important: set an empty array for the mappable
# Create a new axes for the main colorbar. Position it below the suptitle.
# [left, bottom, width, height] as fractions of the figure width/height
cax_main = fig.add_axes([1.05, 0.2, 0.03, 0.6]) # Adjusted position and size for the main colorbar
# Add the colorbar to the main axes
cbar_main = fig.colorbar(mappable, cax=cax_main, orientation='vertical')
cbar_main.ax.tick_params(labelsize=16) # Adjust tick label size for main colorbar

# Add text annotation right next to the main horizontal colorbar
cax_main.text(1.05, 1.07, 'Capacity (GW)', transform=cax_main.transAxes,
              ha='center', va='top', fontsize=16, color='black')


plt.show()
fig.savefig(f"{save_fig_path}/figure_5_new_tx.pdf", dpi=300, bbox_inches='tight')


# Fig 6 - Regional deployment rates VS fossil retirement

In [ ]:
# Renewable rate vs retired fossil cap, regions by colors, 
data = pd.read_excel("/Users/zhenhua/Documents/1_Project_Pathways/0 China_RE_capacity_results.xlsx",
                     sheet_name="Rates")
data = data.iloc[:32, :]

region_list = ['EC', 'CC', 'SC', 'NC', 'NE', 'NW']
region_full_list = ['East', 'Central', 'South', 'North', 'Northeast', 'Northwest']
marker_list = ['circle', 'x', 'triangle-up', 'diamond', 'hexagram', 'cross']

fig = go.Figure()
# for i in range(len(region_list)):
for i in range(28):    
    # remove municipalities
    data = data[~data['provinces'].isin(["Beijing", "Tianjin", "Shanghai", "Chongqing"])].reset_index(drop=True)
    
    x = [data["b1_fossil_retired_2060_actual"].iloc[i]]
    y = [data["b1_renewable_rate"].iloc[i]]
    region = data["grid"].iloc[i]
    province = data["provinces"].iloc[i]
    text_position = data["label_position"].iloc[i]
    i_color = region_list.index(region)
    show_legend = True if i in [13, 10, 26, 16, 20, 24] else False
    fig.add_trace(go.Scatter(x=x, y=y,
                             text=province, textposition=text_position,
                             marker_color=color_order[i_color], name=region_full_list[i_color],
                             mode="markers+text", marker_symbol="circle", 
                             marker_size=15, showlegend=show_legend,
                            ))

# top right
fig.add_annotation(
    text="<b>Large-scale Renewable <br> Deployment</b>",
    align="center",
    showarrow=False,
    xref="paper",
    yref="paper",
    font=font_dict,
    bgcolor="#B6B1A9",
    y=1,
    x=0.5,
    xanchor="center", yanchor="top"
)

# bottom left
fig.add_annotation(
    text="<b>Regional Coordination</b>",
    align="center",
    showarrow=False,
    xref="paper",
    yref="paper",
    font=font_dict,
    bgcolor="#B6B1A9",
    y=0.1,
    x=0.5,
    xanchor="center", yanchor="top"
)


# top left
fig.add_annotation(
    text="<b>Low-carbon Heating / <br> CCS Funding </b>",
    align="center",
    showarrow=False,
    xref="paper",
    yref="paper",
    font=font_dict,
    bgcolor="#B6B1A9",
    y=0.5,
    x=0.1,
    xanchor="center",
)

# bottom right
fig.add_annotation(
    text="<b>Socioeconomic Impact <br> Mitigation</b>",
    align="center",
    showarrow=False,
    xref="paper",
    yref="paper",
    font=font_dict,
    bgcolor="#B6B1A9",
    y=0.5,
    x=0.9,
    xanchor="center",
)

# Add two dashed lines
fig.add_shape( # vertical
    type="line", xref="paper", yref="paper",
    x0=0.5, x1=0.5,  # x-coordinates for the vertical line
    y0=0.15, y1=0.9,  # y-coordinates (start to end)
    line=dict(color="#000000", width=5, dash="dash")
)
fig.add_shape(
    type="line", xref="paper", yref="paper",
    x0=0.2, x1=0.8,  # x-coordinates for the vertical line
    y0=0.5, y1=0.5,  # y-coordinates (start to end)
    line=dict(color="#000000", width=5, dash="dash")
)

fig.update_layout(template="simple_white", barmode='overlay', # or "group"
                  font=font_dict,
                  margin=dict(l=0, r=0, b=0, t=50),
                  uniformtext_minsize=9, uniformtext_mode='show',
                  legend=dict(yanchor="top", y=1.1, xanchor="left", x=0, orientation="h"),
                  legend_title="Grid Region",
                  width=1200, height=800, 
                  xaxis_range=[-40,180], yaxis_range=[-4,24],
                  yaxis_title="Maximum Solar and Wind Deployment Rate (GW/yr)", xaxis_title="Retired Fossil Capacity till 2060 (GW)"
                 )

fig.show()
fig.write_image(f"{save_fig_path}/figure_6a.pdf", scale=12)


# SI 1 - Renewable cost comparison

In [ ]:
# Plot - SI Figure 1
cost = pd.read_csv("/Users/zhenhua/Documents/1_Project_Pathways/2 CCCI_Report/Draft_outline/3 Scenario_designs_sensitivity/Sensitivity_re_capex_cci_2023.csv")
cost_nor = cost[cost["Scenario"] == "Normal"].reset_index(drop=True).set_index("Technology")
cost_con = cost[cost["Scenario"] == "Conservative"].reset_index(drop=True).set_index("Technology")

fig = make_subplots(rows=1, cols=2, shared_yaxes=True,
                    subplot_titles=("Baseline (Optimistic)", "Conservative Cost Declines"))

for tech in ["Onshore wind", "Offshore wind", "Utility-scale solar", 
             "Distributed solar", "Battery", "Pumped hydro", "Long duration (CAES)", "Long duration (VRB)"]:
    marker_symbol = "diamond" if tech == "Long duration (VRB)" else "circle"
    fig.add_trace(go.Scatter(x=cost_nor.T.index[1:], 
                             y=cost_nor.T[tech][1:],  
                             name=tech, marker_symbol=marker_symbol,
                             mode="markers+lines+text", textposition="top center",  marker_size=15,
                             line=dict(color=color_dict[tech], width=2)), row=1, col=1)
    fig.add_trace(go.Scatter(x=cost_con.T.index[1:], 
                             y=cost_con.T[tech][1:],  
                             name=tech, marker_symbol=marker_symbol,
                             mode="markers+lines+text", textposition="top center",  marker_size=15,
                             showlegend=False,
                             line=dict(color=color_dict[tech], width=2)), row=1, col=2)

fig.update_layout(template="simple_white", barmode='group',
                  font=font_dict,
                  margin=dict(l=0, r=0, b=0, t=25),
                  uniformtext_minsize=12, uniformtext_mode='hide',
                  legend=dict(yanchor="top", y=1.0, xanchor="left", orientation="v"),
                  width=1200, height=600, 
                  yaxis_range=[0, 1.1], yaxis_tickformat=".0%")
fig.show()
fig.write_image(f"{save_fig_path}/si_figure_1.pdf", scale=12)


# SI 2 - Learning rate

In [ ]:
# Plot - SI Figure 2
data = pd.read_excel("/Users/zhenhua/Documents/1_Project_Pathways/0 China_RE_capacity_results.xlsx",
                     sheet_name="Renewable_learning_rate")

fig = make_subplots(rows=1, cols=3, shared_yaxes=False, 
                    subplot_titles=("Solar", "Wind", "Storage"),
                    horizontal_spacing=0.05)

# Plot solar capex and LR
fig.add_trace(go.Scatter(x=["2020", "2030", "2040", "2050", "2060"], 
                         y=np.asarray([5300, 4350, 3400, 2450, 1500]),
                         showlegend=True, marker_symbol="circle", marker_size=15,
                         name="Utility-scale solar (Baseline): learning rate 27.4%", mode="markers+lines", marker_color=color_dict["Solar"]),
              row=1, col=1)
fig.add_trace(go.Scatter(x=["2020", "2030", "2040", "2050", "2060"], 
                         y=np.asarray([5300, 4951, 4150.5, 3403.3, 2656.2]),
                         showlegend=True, marker_symbol="cross", marker_size=15,
                         name="Utility-scale solar (Conservative): learning rate 18.5%", mode="markers+lines", marker_color=color_dict["Solar"]),
              row=1, col=1)
fig.add_trace(go.Scatter(x=["2020", "2030", "2040", "2050", "2060"], 
                         y=np.asarray([5300, 4475, 3650, 2825, 2000]),
                         showlegend=True, marker_symbol="circle", marker_size=15,
                         name="Distributed solar (Baseline): learning rate 25.2%", mode="markers+lines", marker_color="#F3E500"),
              row=1, col=1)
fig.add_trace(go.Scatter(x=["2020", "2030", "2040", "2050", "2060"], 
                         y=np.asarray([5300, 5112.7, 4296.8, 3468.9, 2640.9]),
                         showlegend=True, marker_symbol="cross", marker_size=15,
                         name="Distributed solar (Conservative): learning rate 14.5%", mode="markers+lines", marker_color="#F3E500"),
              row=1, col=1)

# Plot wind capex and LR
fig.add_trace(go.Scatter(x=["2020", "2030", "2040", "2050", "2060"], 
                         y=np.asarray([7700,6525,5350,4175,3000]),
                         showlegend=True, marker_symbol="circle", marker_size=15,
                         name="Onshore wind (Baseline): learning rate 23.7%", mode="markers+lines", marker_color=color_dict["Wind"]),
              row=1, col=2)
fig.add_trace(go.Scatter(x=["2020", "2030", "2040", "2050", "2060"], 
                         y=np.asarray([7700,7197.2,6711.4,6225.5,5739.7]),
                         showlegend=True, marker_symbol="cross", marker_size=15,
                         name="Onshore wind (Conservative): learning rate 9.3%", mode="markers+lines", marker_color=color_dict["Wind"]),
              row=1, col=2)
fig.add_trace(go.Scatter(x=["2020", "2030", "2040", "2050", "2060"], 
                         y=np.asarray([15000,12600,10200,7800,5400]),
                         showlegend=True, marker_symbol="circle", marker_size=15,
                         name="Offshore wind (Baseline): learning rate 24.4%", mode="markers+lines", marker_color="#00C6D7"),
              row=1, col=2)
fig.add_trace(go.Scatter(x=["2020", "2030", "2040", "2050", "2060"], 
                         y=np.asarray([15000,13008.3,12256.5,11799.3,11342]),
                         showlegend=True, marker_symbol="cross", marker_size=15,
                         name="Offshore wind (Conservative): learning rate 5.9%", mode="markers+lines", marker_color="#00C6D7"),
              row=1, col=2)

# Plot battery capex and LR
fig.add_trace(go.Scatter(x=["2020", "2030", "2040", "2050", "2060"], 
                         y=np.asarray([6000,5175,4350,3525,2700]),
                         showlegend=True, marker_symbol="circle", marker_size=15,
                         name="Battery (Baseline): learning rate 22.0%", mode="markers+lines", marker_color=color_dict["battery"]),
              row=1, col=3)
fig.add_trace(go.Scatter(x=["2020", "2030", "2040", "2050", "2060"], 
                         y=np.asarray([6000,5644.6,5259,4869.6,4480.2]),
                         showlegend=True, marker_symbol="cross", marker_size=15,
                         name="Battery (Conservative): learning rate 10.9%", mode="markers+lines", marker_color=color_dict["battery"]),
              row=1, col=3)

fig.update_layout(template="simple_white", boxmode='group',
                  font=font_dict,
                  margin=dict(l=0, r=0, b=0, t=25),
                  uniformtext_minsize=12, uniformtext_mode='hide',
                  legend=dict(yanchor="top", y=1.0, xanchor="left", x=1.0, orientation="v", traceorder="normal"),
                  width=1200, height=600, 
                  yaxis_title="Capital Cost (RMB/kW)")

fig.show()  
fig.write_image(f"{save_fig_path}/si_figure_2.pdf", scale=12)
